In [1]:
import os

from scipy import stats
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import yaml
try:
    from yaml import CLoader as Loader
except ImportErorr:
    from yaml import Loader

In [2]:
# Set plot size
factor = 1.2
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*factor

In [3]:
# Folder for data files
data_dir = "data"
# Config file name
cfg_name = "config.yaml"
# Data file name
data_name = "test.csv"

In [4]:
data_path = os.path.join(data_dir, data_name)
cfg_path = os.path.join(data_dir, cfg_name)

# Read config
with open(cfg_path, "r") as f:
    cfg = yaml.load(f, Loader=Loader)
    
# Read data
data = pd.read_csv(data_path)

data["I_in"] = data["I_in"] * 1e6
data["I_meas"] = data["V_i"] / config["r_sense"] / config["gain"] * 1e6
data["V_meas"] = data["V_2x"] / 2.

FileNotFoundError: [Errno 2] No such file or directory: 'data/config.yaml'

In [ ]:
def plot_accuracy(x, y):
    """Plots a graph showing the accuracy of measurements y to ideal values x
    
    Parameters
    ----------
    x : Series
        True values
    y : Series
        Measured values
        
    Returns
    -------
    fig, ax
        Matplotlib figure, same format as plt.subplots
    """
    
    fig, ax = plt.subplots()
    ax.plot(x, y / x)
    ax.axhline(y=1, linestyle=':', color='k', linewidth='3')
    ax.grid()

In [ ]:
fig, ax = plot_accuracy(data["V_in"], V["V_meas"])

ax.set_title("Voltage Channel Accuracy Raw")
ax.set_xlabel("Input Voltage (V)")
ax.set_ylabel("Relative Measurement (V/V)")

In [ ]:
fig, ax = plot_accuracy(data["I_in"], V["I_meas"])

ax.set_title("Current Channel Accuracy Raw")
ax.set_xlabel("Input Current (uA)")
ax.set_ylabel("Relative Measurement (A/A)")

In [ ]:
def calc_fit(x, y):
    """Calculates linear regression fit line given input x and output y.
    
    Parameters
    ----------
    x : Series
        input
    y : Series
        output
        
    Returns
    -------
    Series
        Fit line
    """
    
    linreg = stats.linregress(x, y)
    fit = linreg.intercept + x * linreg.slope
    return fit

In [ ]:
# Linear Regression
data["V_fit"] = calc_fit(data["V_meas"], data["V_in"])
data["I_fit"] = calc_fit(data["I_meas"], data["I_in"])

In [ ]:
fig, ax = plot_accuracy(data["V_in"], V["V_fit"])

ax.set_title("Voltage Channel Accuracy Calibrated")
ax.set_xlabel("Input Voltage (V)")
ax.set_ylabel("Relative Measurement (V/V)")

In [ ]:
fig, ax = plot_accuracy(data["I_in"], V["I_fit"])

ax.set_title("Current Channel Accuracy Calibrated")
ax.set_xlabel("Input Current (uA)")
ax.set_ylabel("Relative Measurement (A/A)")